# U.S. Geological Survey Intro to MODFLOW Class
San Diego, CA\
January 6-10, 2025

## Prototyping Observations for PEST++ with the MV Model

In this notebook, we prototype how to assemble the observations into a single vector and write an instruction file to convey model results to PEST++. The logic of this notebook must get incorporated into the forward run script which we do in the next notebook.

In [1]:
from pathlib import Path

import pandas as pd

In [2]:
# select the folder containing example model observation output
rundir = Path("../pest_obs_prototype/")

In [3]:
# now list the output files - we happen to know the only CSV files
# are the model output files in this dir
outfiles = sorted(list(rundir.glob("*.csv")))
outfiles

[PosixPath('../pest_obs_prototype/at.csv'),
 PosixPath('../pest_obs_prototype/at.wt.csv'),
 PosixPath('../pest_obs_prototype/chd.csv'),
 PosixPath('../pest_obs_prototype/riv.csv')]

In [4]:
# 👆👆👆👆 note that we were explicit about order here,
# because glob may differ by os (IMPORTANT!)
# So using sorted is important


In [5]:
# read all the output files into a single dataframe
obs = pd.concat([pd.read_csv(i).T.iloc[1:] for i in outfiles])
obs.columns = ["obsval"]
obs

,obsval
W01,15.704923
W02,13.327885
W03,16.088156
W04,15.418734
W05,11.974261
W06,13.669466
W07,9.649758
W08,9.273497
W09,9.872218
W10,13.981729


In [6]:
# make the DS observation cumulative, including Pollack Ford
obs.loc["DS"] = obs.loc["DS"] + obs.loc["PF"]
# also calculate a few (3) vertical head difference targets
obs.loc["UW02", "obsval"] = obs.loc["U02", "obsval"] - obs.loc["W02", "obsval"]
obs.loc["UW08", "obsval"] = obs.loc["U08", "obsval"] - obs.loc["W08", "obsval"]
obs.loc["UW15", "obsval"] = obs.loc["U15", "obsval"] - obs.loc["W15", "obsval"]

In [7]:
# write out all these data to an example file
obs.to_csv("../pest_background_files/allobs.dat", sep=" ")

In [8]:
# make the observation names into lower case
obsnames = [i.lower() for i in obs.index.tolist()]
obsnames

['w01',
 'w02',
 'w03',
 'w04',
 'w05',
 'w06',
 'w07',
 'w08',
 'w09',
 'w10',
 'w11',
 'w12',
 'w13',
 'w14',
 'w15',
 'w16',
 'w17',
 'u01',
 'u02',
 'u03',
 'u04',
 'u05',
 'u06',
 'u07',
 'u08',
 'u09',
 'u10',
 'u11',
 'u12',
 'u13',
 'u14',
 'u15',
 'u16',
 'u17',
 'chdflow',
 'pf',
 'ds',
 'uw02',
 'uw08',
 'uw15']

### making the assumption that model output will be concatenated exactly as done above, we can make a simple instruction file that will be used by PEST++ to read model output

In [9]:
with open("../pest_background_files/allobs.dat.ins", "w") as ofp:
    ofp.write("pif ~\n")
    obs0 = obsnames.pop(0)
    ofp.write(f"l2 w !{obs0}!\n")
    [ofp.write(f"l1 w !{i}!\n") for i in obsnames]